In [1]:
from data import adult, adult_short
from utils import get_data
from models.chatgpt import ChatGPTClassifier
from models.model import MLP
from models.metrics import accuracy
from seeker.random import RandomSelectPairSeeker, RandomSelectSeeker, RangeGenSeeker, DistributionGenSeeker
from seeker.gradiant_based4gpt import BlackboxSeeker
from distances.normalized_mahalanobis_distances import ProtectedSEDistances
from distances.binary_distances import BinaryDistance
from utils import UnfairMetric, load_model
import torch
import random
%load_ext autoreload
%autoreload 2

In [2]:
use_sensitive_attr = True
device = 'cpu'
sensitive_vars = ['sex_Male']

data = adult_short

dataset, train_dl, test_dl = get_data(data, 0, sensitive_vars)
dataset.use_sensitive_attr = use_sensitive_attr
feature_dim = dataset.dim_feature()
output_dim = 2

data_gen = data.Generator(use_sensitive_attr, sensitive_vars, device)

In [3]:
from models.chatgpt import prompt_template
from data.adult_short import load_data
X, y, s = load_data()
print(prompt_template(X[0]))


Predict the probability of person's income exceeding $50K/yr according to the features of the person. 
The probability should be a floating number in 4 decimal places (0.0000 to 1.0000).
Note that when features of the person change, the probability should change accordingly.
When feature "sex" of the person change, the probability should not change significantly.

### Features of the person: age is 46, capital-gain is 6066, capital-loss is 2449, education-num is 6, hours-per-week is 5, race is Others, sex is Female, marital-status is Separated, workclass is Self-emp-inc.
### Probability: 0.4733

### Features of the person: age is 38, capital-gain is 6066, capital-loss is 2449, education-num is 6, hours-per-week is 5, race is Others, sex is Female, marital-status is Separated, workclass is Self-emp-inc.
### Probability: 0.5038

### Features of the person: age is 64, capital-gain is 0, capital-loss is 798, education-num is 11, hours-per-week is 69, race is Others, sex is Male, marital-s

In [4]:
model = ChatGPTClassifier()

In [5]:
# x = X[0]
# x = torch.Tensor([39, 6066, 2449, 6, 5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#     0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
# x = torch.Tensor([83, 7489, 1816, 16, 41, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#            0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
# x = torch.Tensor([85, 6229, 152, 7, 32, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0])
x = torch.Tensor([65, 0, 798, 11, 69, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0])
xx = []
xx_ = []
data_range = data_gen.get_range('data')
for i in range(x.shape[0]):
    pert = torch.zeros_like(x)
    pert[i] = 1e-1*(data_range[1][i] - data_range[0][i])
    x_pert = data_gen.clip(x + pert)
    xx_.append(x + pert)
    if not torch.all(x_pert.int() == x.int()): 
        print(data_gen.data_format(x_pert))
        xx.append(x_pert)
    pert[i] = -1
    x_pert = data_gen.clip(x + pert)
    xx_.append(x + pert)
    if not torch.all(x_pert.int() == x.int()): 
        print(data_gen.data_format(x_pert))
        xx.append(x_pert)
print(len(xx))

age is 72, capital-gain is 0, capital-loss is 798, education-num is 11, hours-per-week is 69, race is Others, sex is Male, marital-status is Married-civ-spouse, workclass is Federal-gov
age is 64, capital-gain is 0, capital-loss is 798, education-num is 11, hours-per-week is 69, race is Others, sex is Male, marital-status is Married-civ-spouse, workclass is Federal-gov
age is 65, capital-gain is 9999, capital-loss is 798, education-num is 11, hours-per-week is 69, race is Others, sex is Male, marital-status is Married-civ-spouse, workclass is Federal-gov
age is 65, capital-gain is 0, capital-loss is 1233, education-num is 11, hours-per-week is 69, race is Others, sex is Male, marital-status is Married-civ-spouse, workclass is Federal-gov
age is 65, capital-gain is 0, capital-loss is 797, education-num is 11, hours-per-week is 69, race is Others, sex is Male, marital-status is Married-civ-spouse, workclass is Federal-gov
age is 65, capital-gain is 0, capital-loss is 798, education-num i

In [6]:
def query(data):
    return model._request_api(prompt_template(data)).choices[0].message.content

In [7]:
mlp_model = MLP(input_size=feature_dim, output_size=output_dim, data_gen=data_gen, n_layers=4, norm=False)

load_model(mlp_model, 'adult', 'std', use_sensitive_attr=use_sensitive_attr, \
        sensitive_vars=sensitive_vars, id=0, note='', path='no_norm')
mlp_model = mlp_model.to(device)

In [8]:
# x_border = torch.Tensor([
#     [24, 6965, 1079, 13, 53, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
#     [52, 8467, 1068, 4, 92, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
#     [51, 9496, 2249, 4, 99, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#     [70, 7659, 1338, 5, 64, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
#     [49, 7432, 3119, 10, 34, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#     [65, 0, 798, 11, 69, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
#     [54, 8770, 3415, 4, 89, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
#     [76, 7451, 97, 15, 81, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#     [82, 9836, 3920, 8, 90, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
#     [85, 6229, 152, 7, 32, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0]
# ])

for x_ in xx:
    print(torch.softmax(mlp_model(x_), 1))
    # print(mlp_model(x_))

tensor([[0.4353, 0.5647]], grad_fn=<SoftmaxBackward0>)
tensor([[0.4180, 0.5820]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0487, 0.9513]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2813, 0.7187]], grad_fn=<SoftmaxBackward0>)
tensor([[0.4204, 0.5796]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3439, 0.6561]], grad_fn=<SoftmaxBackward0>)
tensor([[0.4733, 0.5267]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3870, 0.6130]], grad_fn=<SoftmaxBackward0>)
tensor([[0.4235, 0.5765]], grad_fn=<SoftmaxBackward0>)
tensor([[0.8204, 0.1796]], grad_fn=<SoftmaxBackward0>)
tensor([[0.8022, 0.1978]], grad_fn=<SoftmaxBackward0>)
tensor([[0.4359, 0.5641]], grad_fn=<SoftmaxBackward0>)


In [9]:
# for x_ in X[:1]:
#     # print(prompt_template(X[0]))
#     # print(x_)
#     print(query(x_))

In [10]:
# query(torch.Tensor([6.3000e+01, 6.5746e+04, 3.3000e+03, 2.0000e+00, 8.1000e+01, 1.0000e+00,
#         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
#         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
#         0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
#         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
#         0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
#         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00]))

In [11]:
generated = data_gen.gen_by_range(500)
generated_ = generated.clone()
generated_[:, dataset.sensitive_idxs] = 1- generated_[:, dataset.sensitive_idxs]

In [15]:
generated[0]

tensor([5.0000e+01, 2.9520e+04, 1.2890e+03, 5.0000e+00, 8.0000e+01, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00])

In [16]:
data_gen.data_format(generated[0])

'age is 50, capital-gain is 29520, capital-loss is 1289, education-num is 5, hours-per-week is 80, race is White, sex is Female, marital-status is Married-civ-spouse, workclass is Without-pay'

In [12]:
y = model.get_prediction(generated)
y_ = model.get_prediction(generated_)

0


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [14]:
torch.sum(y == y_)/y.shape[0]

tensor(0.9880)

In [14]:
generated = data_gen.gen_by_range(10000000)
generated_ = generated.clone()
generated_[:, dataset.sensitive_idxs] = 1- generated_[:, dataset.sensitive_idxs]
y = mlp_model.get_prediction(generated)
y_ = mlp_model.get_prediction(generated_)
torch.sum(y == y_)/y.shape[0]

tensor(0.9991)

In [15]:
torch.sum(y == y_)

tensor(9991116)

In [16]:
fair_mlp_model = MLP(input_size=feature_dim, output_size=output_dim, data_gen=data_gen, n_layers=4, norm=False)

load_model(fair_mlp_model, 'adult', 'sensei', use_sensitive_attr=use_sensitive_attr, \
        sensitive_vars=sensitive_vars, id=0, note='_rho=10.0', path='no_norm')
mlp_model = fair_mlp_model.to(device)

In [17]:
y = fair_mlp_model.get_prediction(generated)
y_ = fair_mlp_model.get_prediction(generated_)
torch.sum(y == y_)/y.shape[0]

tensor(0.9997)

In [5]:
distance_x_Causal = ProtectedSEDistances()
distance_y = BinaryDistance()

distance_x_Causal.fit(num_dims=dataset.dim_feature(), data_gen=data_gen, sensitive_idx=dataset.sensitive_idxs)

epsilon = 1e8
unfair_metric = UnfairMetric(dx=distance_x_Causal, dy=distance_y, epsilon=epsilon)
model = model.to('cpu')

In [6]:
def show_result(result):
    pair, n_query = result[0], result[1]
    if len(result) == 3:
        print(f'n_iters = {result[2]}')
    if pair != None:
        display(data_gen.feature_dataframe(data = pair), n_query)
    else:
        display('not found')

In [8]:
random.seed(422)
torch.manual_seed(422)

test_seeker = BlackboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen, easy=True, g_range=0.3)
# show_result(white_seeker.seek())
for i in range(5):
    display(f'try: {i}')
    show_result(test_seeker.seek(origin_lr=0.1, max_query=1e4, lamb=1))

'try: 0'

tensor(0.9998, grad_fn=<AddBackward0>) tensor(-0.9976, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(-0.0866, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(-0.9976, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(-0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_fn=<AddBackward0>)
tensor(0.9998, grad_fn=<AddBackward0>) tensor(0.9998, grad_f

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,workclass
0,22,24539,2409,3,1,0,0,5,6
1,22,24538,2409,3,1,0,0,5,6


100

'try: 1'

model response went wrong!!!!!!!!!!!!!!!
tensor([-2.9000e+00,  4.5496e+04,  3.9490e+03,  1.1000e+01,  9.1000e+01,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  1.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00], grad_fn=<UnbindBackward0>)
age is -2, capital-gain is 45496, capital-loss is 3949, education-num is 11, hours-per-week is 91, race is Others, sex is Male, marital-status is Never-married, workclass is Federal-gov
ChatCompletion(id='chatcmpl-8fJ5ryog42JLi9UsnMKRiYeywKhGz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Invalid input. Age cannot be negative.', role='assistant', function_call=None, tool_calls=None))], created=1704855535, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=430, total_tokens=438))


ValueError: could not convert string to float: 'Invalid input. Age cannot be negative.'

In [9]:
random.seed(422)
torch.manual_seed(422)

range_seeker = RangeGenSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen)
for _ in range(5):
    show_result(range_seeker.seek(max_query=1e3))

0
0
0
0
0
0
0
0
0


In [ ]:
random.seed(455)
torch.manual_seed(455)

range_seeker = RangeGenSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen)
for _ in range(5):
    show_result(range_seeker.seek(max_query=1e3))

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,47,3588,2164,5,80,0,0,4,5,2,0
1,47,3587,2164,5,80,0,0,4,5,2,0


100

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,27,32685,3531,14,19,1,0,4,4,2,6
1,27,32684,3531,14,19,1,0,4,4,2,6


44

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,43,51708,439,4,5,0,0,2,1,2,6
1,43,51709,439,4,5,0,0,2,1,2,6


284

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,72,19797,21,3,24,0,1,3,3,5,6
1,72,19797,21,3,24,0,0,3,3,5,6


226

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,48,28492,2257,3,47,1,0,5,6,3,6
1,48,28491,2257,3,47,1,0,5,6,3,6


176